In [1]:
import pandas as pd

confirmed = pd.read_csv('./data/time_series_covid_19_confirmed.csv')
deaths = pd.read_csv('./data/time_series_covid_19_deaths.csv')
recoverd = pd.read_csv('./data/time_series_covid_19_recovered.csv')


In [2]:
# 데이터 정보 확인 
# province/state : 결측값 존재
# Country/Region : 범주형 자료
# lat , long : 위도 경도는 실수형 자료
confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 76 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  80 non-null     object 
 1   Country/Region  258 non-null    object 
 2   Lat             258 non-null    float64
 3   Long            258 non-null    float64
 4   1/22/20         258 non-null    int64  
 5   1/23/20         258 non-null    int64  
 6   1/24/20         258 non-null    int64  
 7   1/25/20         258 non-null    int64  
 8   1/26/20         258 non-null    int64  
 9   1/27/20         258 non-null    int64  
 10  1/28/20         258 non-null    int64  
 11  1/29/20         258 non-null    int64  
 12  1/30/20         258 non-null    int64  
 13  1/31/20         258 non-null    int64  
 14  2/1/20          258 non-null    int64  
 15  2/2/20          258 non-null    int64  
 16  2/3/20          258 non-null    int64  
 17  2/4/20          258 non-null    int

# 데이터 전처리

In [3]:
# 데이터 전처리 (Province/State) 제거
confirmed_1 = confirmed.drop(['Province/State'] ,axis = 1)
deaths_1 = confirmed.drop(['Province/State'] ,axis = 1)
recoverd_1 = recoverd.drop(['Province/State'] ,axis = 1)

In [4]:
# 확진자 , 완치자 , 사망자 데이터를 분석하기 위해 위도 경도 삭제
confirmed_2 = confirmed_1.drop(['Lat','Long'] , axis = 1)
deaths_2= deaths_1.drop(['Lat','Long'] , axis = 1)
recoverd_2 = recoverd_1.drop(['Lat','Long'] , axis = 1)


## 중복된 country 찾고 그룹화 하기

In [5]:
# 중복된 나라 찾기 
import pandas as pd 
Country=confirmed_2.iloc[:,0]
df = Country.value_counts()
df.index[1]

for i in range(len(df)):
    a = df[i]
    if a > 1:
        print(df.index[i])

China
Canada
France
United Kingdom
Australia
Netherlands
Denmark


-중복된 나라
(China
Canada
France
United Kingdom
Australia
Netherlands
Denmark)

In [6]:
# 중복된 나라 그룹화 해서 값을  
confirmed_3 = confirmed_2.groupby('Country/Region').sum()
deaths_3= deaths_2.groupby('Country/Region').sum()
recoverd_3= recoverd_2.groupby('Country/Region').sum()

In [7]:
# 결과 확인
df = confirmed_3.loc[['China',]].T
df

Country/Region,China
1/22/20,548
1/23/20,643
1/24/20,920
1/25/20,1406
1/26/20,2075
...,...
3/29/20,82122
3/30/20,82198
3/31/20,82279
4/1/20,82361


In [8]:
# 그룹화 하기 전 데이터의 sum ()
example = confirmed_2.set_index("Country/Region")
example.loc[['China']].sum()
# 비교 결과 비교 전과 후 의 값은 같다 

1/22/20      548
1/23/20      643
1/24/20      920
1/25/20     1406
1/26/20     2075
           ...  
3/29/20    82122
3/30/20    82198
3/31/20    82279
4/1/20     82361
4/2/20     82432
Length: 72, dtype: int64

# 시각화 

지금 가지고 data  
나라별 확진자 , 사망자 , 완치자 (수)

1. 누적 확진자 전 세계 기준으로 0  

2. 누적 확진자 top 15 

2. 누적 확진자 top 3

선 그래프 시계열로 표현 (사망자 , 완치자 )

+ 전세계 확진자, 사망자 , 완치자 다 더하고 

추세 분석!!

In [18]:
# 4월 2일 기준으로 누적 확진자 데이터
confirmed_4 = confirmed_3.iloc[ : ,  -1 ]
deaths_4 = deaths_3.iloc[ : ,  -1 ]
recoverd_4 = recoverd_3.iloc[ : ,  -1 ]

In [10]:
df = confirmed_4.sort_values()
confirmed_5

Country/Region
Turkey             18135
Switzerland        18827
United Kingdom     34173
Iran               50468
France             59929
China              82432
Germany            84794
Spain             112065
Italy             115242
US                243453
Name: 4/2/20, dtype: int64

## 전세계 covid - 19

In [28]:
import plotly.graph_objects as go
import plotly.express as px
fig  = go.Figure()
fig = px.bar(confirmed_4, x = confirmed_4.index , y = confirmed_4.values, 
              labels = {'covid - 19 : confirmed'} , height = 600)
fig.show()

In [25]:
import plotly.graph_objects as go

fig = go.Figure(
data = [go.Bar(name = 'comfirmed', x = confirmed_4.index , y = confirmed_4.values ),
       go.Bar(name = 'deaths' , x = deaths_4.index , y = deaths_4.values ),
       go.Bar(name = 'recoverd', x =recoverd_4.index, y = recoverd_4.values)])
fig.update_layout(barmode = 'stack' , title_text = '4월 2일 코로나 확진자, 사망자, 완치자')
fig.show()

## 확진자 top 15

In [45]:
# 누적 확진자 top 15를 기준으로 사망자와 완치자를 나라에 맞게 따로 구해야한다 
confiremd_5 = confirmed_4.sort_values().tail(15)
a = []
for i in confiremd_5.index:
    a.append(i)


list

In [44]:
deaths[a]

KeyError: "None of [Index(['Korea, South', 'Austria', 'Canada', 'Netherlands', 'Belgium', 'Turkey',\n       'Switzerland', 'United Kingdom', 'Iran', 'France', 'China', 'Germany',\n       'Spain', 'Italy', 'US'],\n      dtype='object')] are in the [columns]"